In [43]:
!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ericschlosser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [44]:
from nltk import pos_tag, word_tokenize
pos_tag(word_tokenize("John's big idea isn't all that bad."))

[('John', 'NNP'),
 ("'s", 'POS'),
 ('big', 'JJ'),
 ('idea', 'NN'),
 ('is', 'VBZ'),
 ("n't", 'RB'),
 ('all', 'PDT'),
 ('that', 'DT'),
 ('bad', 'JJ'),
 ('.', '.')]

In [45]:
!pip install newspaper3k 


In [46]:
import os, ssl
if(not os.environ.get('PYTHONHTTPVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
import nltk
dler = nltk.downloader.Downloader()
dler._update_index()
nltk.download('punkt')
dler._status_cache['panlex_lite'] = 'installed'
from newspaper import Article
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ericschlosser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
def scan_art(url):
    article = Article(url, language = 'en')
    article.download()
    article.parse()
    article.nlp()
    artlist = article.text.split()
    letters = "abcdefghijklmnopqrstuvwxyz-'"
    artlist1 = []
    for word in artlist:
        for character in word:
            if character.lower() not in letters:
                word = word.replace(character, '')
        artlist1.append(word)
    df = pd.read_csv('/Users/ericschlosser/Desktop/TheHardWay/Wordle/bnc.csv')
    # We're going to have to change the way diff level is input.
    df1 = df[df['Difficulty'] >= 8]
    #df_gre = pd.read_csv('/Users/ericschlosser/Desktop/TheHardWay/Wordle/bnc.csv')


    word_list = []

    for word in artlist1:
        if word.upper() in df1['Branch'].values:
            word_list.append(df1[df1['Branch'] == word.upper()]['Root'].values[0])

    word_list = set(word_list)
    word_list = list(word_list)
    
    return word_list
    #df2 = pd.DataFrame(word_list, columns= ['Words'])
    #df2['Words'] = pd.DataFrame(word_list)
    #wlpath = os.path.join(current_app.root_path, 'wordlists', str(self.username) + '_article_list.pkl')
    #df2.to_pickle(wlpath, compression = None)

In [48]:
word_list = scan_art("https://www.newyorker.com/news/daily-comment/the-new-nuclear-reality")

In [49]:
type(word_list)

list

In [50]:
def ner(url):
    article = Article(url, language = 'en')
    article.download()
    article.parse()
    text = ""
    paragraphs = article.text.split('\n')
    #article.nlp()
    old_p = []
    for paragraph in paragraphs:
        new_p = pos_tag(word_tokenize(paragraph))
        old_p = old_p +new_p
    df_pos = pd.DataFrame(old_p, columns = ['Word', 'PoS'])
    df_pos['Word'] = df_pos['Word'].str.upper()
    return df_pos

In [51]:
df_pos = ner("https://www.newyorker.com/news/daily-comment/the-new-nuclear-reality")

In [52]:
df_pos.head()

,Word,PoS
0,IN,IN
1,HIS,PRP$
2,NOBEL,NNP
3,PEACE,NNP
4,PRIZE,NNP


In [54]:
df = df_pos[df_pos['Word'].isin(word_list)]

In [57]:
df.shape

(14, 2)

In [61]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ericschlosser/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [62]:

import nltk
from nltk.corpus import wordnet as wn

In [65]:
df

,Word,PoS
132,DECIMATE,VB
160,LEXICON,NN
275,WEAPONRY,NN
414,HYPERSONIC,JJ
480,INTERCONTINENTAL,JJ
626,IMPERILLED,VBN
853,ARSENAL,JJ
965,PENTAGON,NNP
1043,ARSENAL,NN
1204,TENUOUS,JJ


In [92]:
syns = wn.synsets("imperil", pos = 'v')
for defi in syns:
    print(defi.definition())

pose a threat to; present a danger to


In [69]:
for word in df['Word'].values:
    print(df[df['Word'] == word].iat[0,1])

VB
NN
NN
JJ
JJ
VBN
JJ
NNP
JJ
JJ
JJ
NN
JJ
NNP


In [80]:
def pos_def(df):
    df_pos = pd.DataFrame(columns = ['Word', 'PoS', 'Def'])
    for word in df['Word'].values:
        def_list = []
        pos_var = df[df['Word'] == word].iat[0,1]
        if pos_var in ['VB', 'VBN']:
            pos_var = 'v'
        elif pos_var in ['JJ']:
            pos_var = 'a'
        elif pos_var in ['NN', 'NNP']:
            pos_var = 'n'
        syns = wn.synsets(word, pos = pos_var)
        for defi in syns:
            def_list.append(defi.definition())
        for definition in def_list:
            df_pos.loc[len(df_pos)] = [word, pos_var, definition]
            
    return df_pos

In [82]:
df_pos = pos_def(df)
df_pos

,Word,PoS,Def
0,DECIMATE,v,"kill one in every ten, as of mutineers in Roma..."
1,DECIMATE,v,kill in large numbers
2,LEXICON,n,a language user's knowledge of words
3,LEXICON,n,a reference book containing an alphabetical li...
4,WEAPONRY,n,weapons considered collectively
5,INTERCONTINENTAL,a,extending or taking place between or among con...
6,PENTAGON,n,a government building with five sides that ser...
7,PENTAGON,n,the United States military establishment
8,PENTAGON,n,a five-sided polygon
9,TENUOUS,a,having thin consistency


In [83]:
def pos_def(df):
    df_pos = pd.DataFrame(columns = ['Word', 'PoS', 'Def'])
    for word in df['Word'].values:
        def_list = []
        pos_var = df[df['Word'] == word].iat[0,1]
        if pos_var in ['VB', 'VBN']:
            pos_var = 'v'
        elif pos_var in ['JJ']:
            pos_var = 'a'
        elif pos_var in ['NN', 'NNP']:
            pos_var = 'n'
        syns = wn.synsets(word, pos = pos_var)
        definition = ''
        for defi in syns:
            definition = definition + defi.definition() + ' OR '
        df_pos.loc[len(df_pos)] = [word, pos_var, definition[:-4]]
            
    return df_pos

In [84]:
df_pos = pos_def(df)
df_pos

,Word,PoS,Def
0,DECIMATE,v,"kill one in every ten, as of mutineers in Roma..."
1,LEXICON,n,a language user's knowledge of words OR a refe...
2,WEAPONRY,n,weapons considered collectively
3,HYPERSONIC,a,
4,INTERCONTINENTAL,a,extending or taking place between or among con...
5,IMPERILLED,v,
6,ARSENAL,a,
7,PENTAGON,n,a government building with five sides that ser...
8,ARSENAL,a,
9,TENUOUS,a,having thin consistency OR very thin in gauge ...


In [86]:
df_bnc = pd.read_csv('/Users/ericschlosser/Desktop/TheHardWay/Wordle/bnc.csv')

In [87]:
df_bnc.head()

,Unnamed: 0,Root,Branch,Difficulty
0,0,A,A,1
1,1,A,AN,1
2,2,ABLE,ABLE,1
3,3,ABLE,ABILITIES,1
4,4,ABLE,ABILITY,1


In [88]:
df_bnc[df_bnc['Branch'] == 'HYPERSONIC']

,Unnamed: 0,Root,Branch,Difficulty
65298,65298,HYPERSONIC,HYPERSONIC,19


In [89]:
syns = wn.synsets("hypersonic")
for defi in syns:
    print(defi.definition())

In [90]:
print(syns)

[]
